In [1]:
import pandas as pd
import gc
#preprocessing
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
#from sklearn.preprocessing import LabelEncoder

#layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D, SimpleRNN, GlobalMaxPooling1D
from keras.layers.wrappers import Bidirectional

#helper libraries
from keras import metrics
import h5py
from keras.models import model_from_json
import pickle

c:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Model architecture from https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/download/9745/9552 

by Siwei Lai, Liheng Xu, Kang Liu, Jun Zhao

In [2]:
dat = pd.read_csv('G:\\Script\\LeadIQ\\train.csv',encoding='latin')

In [3]:
#data shape
dat.shape

(99989, 3)

In [4]:
#sample data from top rows
dat.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [5]:
#sample data from bottom rows
dat.tail()

,ItemID,Sentiment,SentimentText
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...
99988,100000,1,@cupcake_kayla haha yes you do


In [6]:
#unique values in sentiment label
dat.Sentiment.unique()

array([0, 1], dtype=int64)

In [7]:
#####clean text
#twitter handle name
dat.SentimentText=dat.SentimentText.str.replace(pat='(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',repl='')
#multiple space
dat.SentimentText=dat.SentimentText.str.replace(pat='[ ]{2,}',repl=' ')

In [8]:
#split data into test and train
###divide data for traininng and test
dat_train = dat[:90000]
dat_test = dat[90000:]

In [9]:
#tokenization
texts_train = dat_train.SentimentText
texts_test = dat_test.SentimentText

In [10]:
###train tokenizer
num_max = 1000
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts_train)

In [11]:
num_max = 1000
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts_train)

####save tokenizer
with open('G:\\Script\\LeadIQ\\tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#labels
tags_train = dat_train.Sentiment
tags_test = dat_test.Sentiment

In [13]:
#free memory
del dat
gc.collect()
dat=pd.DataFrame()

del dat_train
gc.collect()
dat_train=pd.DataFrame()

del dat_test
gc.collect()
dat_test=pd.DataFrame()

In [14]:
#for rnn preproces train
max_len = 100
rnn_texts_seq_train = tok.texts_to_sequences(texts_train)
rnn_texts_mat_train = sequence.pad_sequences(rnn_texts_seq_train,maxlen=max_len)

#for rnn preproces test
rnn_texts_seq_test = tok.texts_to_sequences(texts_test)
rnn_texts_mat_test = sequence.pad_sequences(rnn_texts_seq_test,maxlen=max_len)

In [15]:
#Embedding
max_features = 5000
maxlen = 100
embedding_size = 128

In [16]:
## model architecture from https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/download/9745/9552
## by Siwei Lai, Liheng Xu, Kang Liu, Jun Zhao
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dense(256, activation='tanh'))
model.add(Bidirectional(SimpleRNN(256, return_sequences = True, activation="tanh", dropout=0.2)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='sgd',metrics=['binary_accuracy'])

In [17]:
model.fit(rnn_texts_mat_train,tags_train,batch_size=15,epochs=2,verbose=0,validation_split=0.2)

In [18]:
#save model structure
model_json = model.to_json()
model_backup=model
with open("G:\\Script\\LeadIQ\\model.json", "w") as json_file:
    json_file.write(model_json)

In [19]:
#save model weights
model.save_weights("G:\\Script\\LeadIQ\\model.h5")
print("Saved model to disk")

Saved model to disk


#### Model performance on test data

 - Model accuracy is very low at 59.95%. I couldn't run more epochs and increase batch size due to lack of computing power. My laptop has 8 GB RAM and no GPU. Everything it predicts is 1.

In [20]:
scores = model.evaluate(rnn_texts_mat_test,tags_test,verbose=0)

In [21]:
print("Model accuracy on test data is",scores[1]*100,"percent")

Model accuracy on test data is 59.94594053578145 percent
